In [2]:
import cv2
import numpy as np
from math import ceil,floor

In [3]:
vidObj = cv2.VideoCapture('input.mov') 

frames = []
input_frames = []

count = 0
success = 1

while success: 
    success, image = vidObj.read()     
    if image is None:
        break
    
    frames.append(image)
    input_frames.append(cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)/255) # grayscale conversion

In [5]:
def ExhaustiveSearch(t,r):
    M,N = r.shape[0],r.shape[1]
    I,J = t.shape[0],t.shape[1]
    
    count = 0
    
    best_node = [0,0]
    max_correlation = 10e10
    
    for m in range(I-M+1): #
        for n in range(J-N+1): #
            current_correlation = 0
            for i in range(m,m+M-1):
                for j in range(n,n+N-1):
                    current_correlation += t[i][j]*r[i-m][j-n]
                    
            if(m%100==0):
                if(n%100==0):
                    print(m,n,current_correlation)
            
            if(current_correlation<max_correlation):
                max_correlation = current_correlation
                best_node = [m,n]
    
    return best_node,max_correlation

In [6]:
reference = cv2.imread('reference.jpg', cv2.IMREAD_GRAYSCALE)/255

In [7]:
#[X,Y],max_correlation = ExhaustiveSearch(input_frames[0],reference)
[X,Y],max_correlation = [247,137], 3588.946220684356
M,N = reference.shape[0],reference.shape[1]

print(X,Y,max_correlation)

247 137 3588.946220684356


In [8]:
def ExhaustiveSearchUsingCenter(t,r,X,Y,p):
    M,N = r.shape[0],r.shape[1]
    I,J = t.shape[0],t.shape[1]
    
    count = 0
    
    best_node = [0,0]
    max_correlation = 10e10
    nos = 0
    
    for m in range(X-p,X+p): # O(2p + 1)
        for n in range(Y-p,Y+p): # O(2p + 1) 
            if m >= I-M or n >= J-N:
                continue
            
            current_correlation = 0
            nos += 1
            for i in range(m,m+M-1): # O(M) 
                for j in range(n,n+N-1): # O(N)
                    current_correlation += t[i][j]*r[i-m][j-n]
            '''   
            if(previous_correlation >= current_correlation):
                max_correlation = current_correlation
                best_node = [m,n]
                break
            '''
            
            if(current_correlation<max_correlation):
                max_correlation = current_correlation
                best_node = [m,n]
                
    return best_node,max_correlation,nos

In [9]:
def twoDimensionalLogarithmicSearch(t,r,X,Y,p):
    M,N = r.shape[0],r.shape[1]
    I,J = t.shape[0],t.shape[1]
    
    count = 0
    
    best_node = [X,Y]
    
    max_correlation = 10e10
    
    div = p
    best_sum = 0
    
    while True:
        arr = [floor(-div/2),0,ceil(div/2)]
        #print(max_correlation)
        cur = 0 
        best = 1
        for m in arr:
            for n in arr: 
                x,y = X+m,Y+n
                
                if x >= I-M or y >= J-N:
                    continue
                
                cur += 1
                current_correlation = 0
                
                for i in range(x,x+M-1):
                    for j in range(y,y+N-1):
                        current_correlation += t[i][j]*r[i-x][j-y]
                
                if(current_correlation<max_correlation):
                    max_correlation = current_correlation
                    best_node = [x,y]
                    best = cur
                
        [X,Y] = best_node
        best_sum += best
        div = int(div/2)
        
        if(div<1):
            break
    
    return best_node,max_correlation,np.log2(best_sum)


In [10]:
def outline_creator(frame,center_x,center_y,M,N):
    for i in range(center_x-5,center_x+M+5):
        frame[i][center_y] = [0,0,255]
        frame[i][center_y+N+5] = [0,0,255]
    
    for j in range(center_y,center_y+N+5):
        frame[center_x-5][j] = [0,0,255]
        frame[center_x+M+5][j] = [0,0,255]
            
    return frame    

In [11]:
def output_frames_ExhaustiveSearch(frames,input_frames,reference,X,Y,p,first_frame_output):
    output = []
    output.append(first_frame_output)
    M,N = reference.shape[0],reference.shape[1]
    
    total = 0
    
    for i in range(1,len(frames)):
        [X,Y],max_correlation,best = ExhaustiveSearchUsingCenter(input_frames[i],reference,X,Y,p)
        total += best
        print(X,Y)
        output.append(outline_creator(frames[i],X,Y,M,N))
    
    return output,int(total/(len(frames)-1))

In [12]:
#twoDimensionalLogarithmicSearch(input_frames[1],reference,247,137,7)

def output_frames_2DLogSearch(frames,input_frames,reference,X,Y,p,first_frame_output):
    output = []
    output.append(first_frame_output)
    M,N = reference.shape[0],reference.shape[1]
    
    total = 0
    
    for i in range(1,len(frames)):
        [X,Y],max_correlation,best = twoDimensionalLogarithmicSearch(input_frames[i],reference,X,Y,p)
        total += best
        output.append(outline_creator(frames[i],X,Y,M,N))
    
    return output,int(total/(len(frames)-1))

In [13]:
def filter_scale(img,scale,filter=False):
    width = int(img.shape[1]/ scale)
    height = int(img.shape[0]/ scale)
    dim = (width, height)

    if filter:
        img = cv2.GaussianBlur(img,(5,5),0)
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        #converted = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
        return resized
    else:
        return img

In [14]:
def hierarchicalSearch(t,r,X,Y,p):
    frames = []
    refs = []
    
    for i in range(3):
        scale = pow(2,i)
        
        if(scale>1):
            t = filter_scale(t,scale,filter=True) # level 0 -> 1 -> 2
            r = filter_scale(r,scale,filter=True) # level 0 -> 1 -> 2
            frames.append(t)
            refs.append(r)
        else:
            frames.append(t)
            refs.append(r)

    # Level 2
    
    [x1,y1],max_correlation,best_level_2 = twoDimensionalLogarithmicSearch(frames[2],refs[2],X//4,Y//4,p//4)
    
    [x1,y1] = [x1-X//4,y1-Y//4] # remove bias values
    
    # Level 1
    
    [x2,y2],max_correlation,best_level_1 = twoDimensionalLogarithmicSearch(frames[1],refs[1],(X//2)+2*x1,(Y//2)+2*y1,p//2)
    
    [x2,y2] = [x2-(X//2),y2-(Y//2)]

    # Level 0
    
    best_node,max_correlation,best_level_0 = twoDimensionalLogarithmicSearch(frames[0],refs[0],(X)+2*x2,(Y)+2*y2,p)
    
    best = best_level_0 + best_level_1 + best_level_2
    
    return best_node,max_correlation,int(best)

def output_frames_hierarchicalSearch(frames,input_frames,reference,X,Y,p,first_frame_output):
    output = []
    output.append(first_frame_output)
    M,N = reference.shape[0],reference.shape[1]
    
    total = 0
    
    for i in range(1,len(frames)):
        [X,Y],max_correlation,best = hierarchicalSearch(input_frames[i],reference,X,Y,p)
        #print(X,Y)
        total += best
        output.append(outline_creator(frames[i],X,Y,M,N))
    
    return output,int(total/(len(frames)-1))

In [22]:
first_frame_output = outline_creator(frames[0],X,Y,M,N)
print(X,Y)
# file = open("Performance.txt","w")
p_vals = [3]

for p in p_vals:
    # output_frames,p_es = output_frames_ExhaustiveSearch(frames,input_frames,reference,X,Y,p,first_frame_output)
    # output_frames,p_2dlog = output_frames_2DLogSearch(frames,input_frames,reference,X,Y,p,first_frame_output)
    output_frames,p_hirar = output_frames_hierarchicalSearch(frames,input_frames,reference,X,Y,p,first_frame_output)
    #file.write(str(p)+str(p_es)+str(p_2dlog)+str(p_hirar)+"\n")
    
# file.close()
# EXHAUSTIVE

# output_frames = output_frames_ExhaustiveSearch(frames,input_frames,reference,X,Y,2,first_frame_output,max_correlation)

# 2D LOGARITHMIC

# output_frames = output_frames_2DLogSearch(frames,input_frames,reference,X,Y,20,first_frame_output)

#output_frames = output_frames_hierarchicalSearch(frames,input_frames,reference,X,Y,20,first_frame_output)

247 137


In [23]:
# outpuut generation

cap = cv2.VideoCapture('input.mov')
fourcc = cv2.VideoWriter_fourcc(*'MPEG')
out = cv2.VideoWriter('output.avi', fourcc, 760/13, (int(cap.get(3)),int(cap.get(4))))


for frame in output_frames:
    out.write(frame)
cap.release()
out.release()